## Langchain + OpenAI LLM tracing using Maxim + Langchain

In [ ]:
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

In [2]:
import os
import logging

LOG_REPOSITORY_ID = os.getenv("MAXIM_LOG_REPO_ID")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Set global debug level for logging
# Options: DEBUG, INFO, WARNING, ERROR, CRITICAL
# logging.basicConfig(level=logging.DEBUG)

In [ ]:
from maxim import Maxim

logger = Maxim().logger()

## Chain with stream

In [ ]:
import time
import json

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import ChatPromptTemplate
from maxim.logger.langchain import MaximLangchainTracer

langchain_tracer = MaximLangchainTracer(logger)

MODEL_NAME = "gpt-4o-mini"
llm = ChatOpenAI(model=MODEL_NAME, api_key=OPENAI_API_KEY,streaming=True)

# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant."), ("human", "{input}")]
)

# Create a chain using pipe
chain = prompt | llm | JsonOutputParser()

# Run the chain with callbacks for tracing
user_input = (
    "give me a basic json response. dont add anything else, just the json. not even ```"
)
response = ""
chunk_index = 0
start_time = time.time()
for event in chain.stream(
    input=user_input,
    config={
        "callbacks": [langchain_tracer],
        "metadata":{"maxim":{"trace_tags": {"test":"test-123"}}}
    },
):
    if chunk_index == 0:
        time_to_first_chunk = time.time() - start_time
    response = json.dumps(event)
    last_chunk = json.dumps(event)
    chunk_index += 1
time_to_last_chunk = time.time() - start_time
logger.flush()
print(response)

## Chain with invoke

In [ ]:
import time
import json

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import ChatPromptTemplate
from maxim.logger.langchain import MaximLangchainTracer

langchain_tracer = MaximLangchainTracer(logger)

MODEL_NAME = "gpt-4o-mini"
llm = ChatOpenAI(model=MODEL_NAME, api_key=OPENAI_API_KEY,streaming=True)

# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant."), ("human", "{input}")]
)

# Create a chain using pipe
chain = prompt | llm | JsonOutputParser()

# Run the chain with callbacks for tracing
user_input = (
    "give me a basic json response. dont add anything else, just the json. not even ```"
)
response = ""
chunk_index = 0
start_time = time.time()
response = chain.invoke(
    input=user_input,
    config={
        "callbacks": [langchain_tracer],
        "metadata":{"maxim":{"trace_tags": {}}}
    },
)

logger.flush()
print(response)